# **Leafmap for geospatial visualisations**

#### Import the necessary packages

In [ ]:
import os
import leafmap.leafmap as leafmap
from area import area
import pyvista as pv

#### We will create and interactive map. Below we set our center, zoom level and map window size. A certain basemap has been set (CyclOSM) ;-), but after the map has loaded, you can browse and change the basemap by clicking on the tool icon in the top right corner and subsequently on the map symbol (top left).

In [ ]:
m = leafmap.Map(center=(47.3966, 8.5494), zoom=16, height="800px", width="100%")
m.add_basemap("CyclOSM")
m

#### After playing with different basemaps, let's set it to "CartoDB.Positron" for a clean background. Now move your map to Darnah on the coast of Libya and open the toolbox again. Click on the airplane symbol to open "OpenAerialMap", which currently holds over 15'000 free UAV and high res satellite images, including some open disaster response data from Maxar. In the input window, leave all fields as they are and press search. You should see some high res satellite imagery and you can inspect the metadata. Check also the layer options through the layer symbol that pops up next to the toolbox symbol.

#### You might want a list of available images. Let's try it with code and create a geodataframe of the results.

In [ ]:
bbox = [22.5, 32, 23, 33]
gdf = leafmap.oam_search(
    bbox=bbox, return_gdf=True
)
print(f"Found {len(gdf)} images")

In [ ]:
gdf.head(2)

In [ ]:
images = gdf["uuid"].tolist()
images[:4]

#### Scientific publications in the field of remote sensing often show image comparisons: the ultimate thing to visualize in a complementary notebook. For convenience, let's use the image links that we just created above. We can do all this without even downloading the data!

In [ ]:
url1 = "https://oin-hotosm.s3.us-east-1.amazonaws.com/6502980d0906de000167e681/0/6502980d0906de000167e682.tif"
url2 = "https://oin-hotosm.s3.us-east-1.amazonaws.com/6502b20a0906de000167e691/0/6502b20a0906de000167e692.tif"
m.split_map(
    left_layer=url1, 
    right_layer=url2
    )
m

#### Now, back to tracing our iceberg. Measuring would be cool, so outsiders can actually verify those claims about the size of icebergs! I've preselected the right image, there is no need to search.

In [ ]:
url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-1-grd"
collection = "sentinel-1-grd"
item = "S1B_EW_GRDH_1SDH_20191213T074429_20191213T074529_019343_024886"

m = leafmap.Map()
m.add_basemap("Esri.OceanBasemap")
m.add_stac_layer(
    collection=collection,
    item=item,
    assets=["hh"],
    name="A68a as seen by Sentinel-1",
)
m

#### Feel free to use the drawing tools on the left hand side and follow the outlines of A68a. You can also directly measure the length of the side we saw in the video.

In [ ]:
geom = m.user_roi
iceberg_polygon = geom["geometry"]
area(iceberg_polygon)
print(f"Area of the iceberg: {area(iceberg_polygon):,.0f} square meters")

#### Sometimes, it is nice to show multiple maps with different layers or different timestamps to allow inspection by the reader...

In [ ]:
layers = [str(f"NLCD {year} CONUS Land Cover") for year in [2001, 2006, 2011, 2016]]
labels = [str(f"NLCD {year}") for year in [2001, 2006, 2011, 2016]]
leafmap.linked_maps(
    rows=2,
    cols=2,
    height="300px",
    layers=layers,
    labels=labels,
    center=[36.1, -115.2],
    zoom=9,
)

#### Thanks to Jasmin her data, we can also inspect and visualize Lidar data, in 3D. Difficult to do this in publications, therefor perfect additional Notebook material!

In [ ]:
file = '/Users/jochem/GitHub/BraakhekkeUZH/timelapse/tile_10_30_normd9eae1d66b5be10ee626dea1aceed548ac28e5cd4560cc2bd7392f31e962be1e.laz'
las = leafmap.read_lidar(file)

In [ ]:
las.header.point_count

In [ ]:
list(las.point_format.dimension_names)

In [ ]:
pv.set_jupyter_backend('trame')
prop = pv.Property()
prop.point_size = 2
prop.style = "points"

In [ ]:
leafmap.view_lidar(file, cmap="spectral", backend="pyvista")

#### And as a last teaser for all geographers, an interactive geojson visualized right here.

In [ ]:
m = leafmap.Map(center=[0, 0], zoom=2)
url = "https://raw.githubusercontent.com/opengeos/leafmap/master/examples/data/countries.geojson"
style = {"fillOpacity": 0.5}
m.add_geojson(
    url,
    layer_name="Countries",
    style=style,
    fill_colors=["red", "yellow", "green", "orange"],
)
m

## Thanks for your participation and play on!